https://youtu.be/PmWVbpaQxqU

Code

In [ ]:
from machine import Pin                # 從machine模組引入Pin物件，用來控制GPIO腳位
from time import ticks_us, sleep_us   # 引入微秒等級的時間函數
import time                           # 引入time模組，使用 sleep() 函數

# 設定超音波模組的腳位
trig = Pin(5, Pin.OUT)    # Trig為輸出腳，接ESP32的GPIO 5
echo = Pin(18, Pin.IN)    # Echo為輸入腳，接ESP32的GPIO 18

# 定義取得距離的函式
def get_distance():
    trig.value(0)         # 確保Trig為低電位
    sleep_us(2)           # 等待2微秒穩定

    trig.value(1)         # 發出10微秒的高電位脈衝
    sleep_us(10)
    trig.value(0)         # 關閉Trig，準備接收回波

    timeout = 10000       # 設定等待超時時間（單位：微秒）

    # 等待Echo變為高電位（開始計時）
    start = ticks_us()
    while echo.value() == 0:
        if ticks_us() - start > timeout:
            return -1     # 超過等待時間，表示無法偵測到物體

    ts = ticks_us()       # 回波開始的時間戳記

    # 等待Echo回到低電位（結束計時）
    start = ticks_us()
    while echo.value() == 1:
        if ticks_us() - start > timeout:
            return -1     # 超過等待時間，表示回波異常或物體太近

    te = ticks_us()       # 回波結束的時間戳記

    tc = te - ts          # 計算回音時間（單位：微秒）

    # 距離計算公式：距離 = 音速 * 時間 / 2
    # 音速約為340公尺/秒，換算成公分/微秒為0.034 cm/us，所以這邊直接除以10000即可
    distance = (tc * 340 / 2) / 10000  # 計算出距離（單位：公分）
    return distance

# 主程式迴圈：不斷取得距離並顯示
while True:
    d = get_distance()    # 取得目前測得的距離
    if d == -1:
        print("太近或沒偵測到物體")  # 顯示錯誤訊息
    else:
        print("距離: {:.2f} 公分".format(d))  # 顯示測得的距離
    time.sleep(0.5)       # 每0.5秒偵測一次
